# Distribution of Oklahoma Travel Information Centers (TICs)

### Business Problem

The Oklahoma Tourism Department would like to study the efficacy of their travel information centers or TICs. In this project, I will determine whether Oklahoma's Travel Information Centers (TICs) are amply spaced throughout the state and whether there are opportunity zones for additional TICs that could better serve tourists.

### Data

I will plot Oklahoma's TICs using lat-long and venue data sourced from Foursquare's Places API.

# Part 1: Installing libraries, importing the dataset, converting to a dataframe

In [11]:
## First I will install the necessary libraries. 
import requests
import pandas as pd 
import numpy as np 
import random 

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html

# tranforming json file into a pandas dataframe
from pandas.io.json import json_normalize

# plotting library
!pip install folium
import folium 

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

#Matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
## Import Foursquare credentials

CLIENT_ID = 'FKNYLWC40IUCVAZ3W0R4NCEWZLBJ0LETV2Z0BOKISGS2TF0W'
CLIENT_SECRET = 'KUQJUS0MXHHEKDOS3NBYZQIQB5WE0S3CY41H1TNWNBNAUSBV'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FKNYLWC40IUCVAZ3W0R4NCEWZLBJ0LETV2Z0BOKISGS2TF0W
CLIENT_SECRET:KUQJUS0MXHHEKDOS3NBYZQIQB5WE0S3CY41H1TNWNBNAUSBV


In [13]:
## Defining an instance of the geocoder
address = 'Oklahoma City, OK 73102'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.4653376 -97.5128162


In [29]:
## Search for the category Travel Information Centers from Foursquare's API

search_query = 'Tourism Information Center'
radius = 70000000
print(search_query + ' .... OK!')

Tourism Information Center .... OK!


In [30]:
## Define GET url

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FKNYLWC40IUCVAZ3W0R4NCEWZLBJ0LETV2Z0BOKISGS2TF0W&client_secret=KUQJUS0MXHHEKDOS3NBYZQIQB5WE0S3CY41H1TNWNBNAUSBV&ll=35.4653376,-97.5128162&v=20180604&query=Tourism Information Center&radius=70000000&limit=30'

In [31]:
## Send the GET request

results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5fadf13971cf84332278296a'},
 'response': {'venues': [{'id': '4c0692edb481d13a3c5819f0',
    'name': 'Downtown OKC Tourism Info Center',
    'location': {'address': '120 N Robinson, Ste 115',
     'crossStreet': 'Park and Robinson',
     'lat': 35.468561,
     'lng': -97.516351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 35.468561,
       'lng': -97.516351}],
     'distance': 481,
     'postalCode': '73102',
     'cc': 'US',
     'city': 'Oklahoma City',
     'state': 'OK',
     'country': 'United States',
     'formattedAddress': ['120 N Robinson, Ste 115 (Park and Robinson)',
      'Oklahoma City, OK 73102',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ff941735',
      'name': 'Miscellaneous Shop',
      'pluralName': 'Miscellaneous Shops',
      'shortName': 'Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
       'suffix': '.png'},
      'primary': True}],
    'referra

In [32]:
## Transform to dataframe

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4c0692edb481d13a3c5819f0,Downtown OKC Tourism Info Center,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1605235001,False,"120 N Robinson, Ste 115",Park and Robinson,35.468561,-97.516351,"[{'label': 'display', 'lat': 35.468561, 'lng':...",481,73102,US,Oklahoma City,OK,United States,"[120 N Robinson, Ste 115 (Park and Robinson), ..."
1,569ca86b498e7bd114de7689,Eden Bail Bond Information Center,[],v-1605235001,False,805 Robert S Kerr Ave,NaN,35.470381,-97.526023,"[{'label': 'display', 'lat': 35.470381, 'lng':...",1322,73106,US,Oklahoma City,OK,United States,"[805 Robert S Kerr Ave, Oklahoma City, OK 7310..."
2,53c576a6498eccdf09559718,Oklahoma Tourism and Recreation Distribution ...,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1605235001,False,210 NE 31st St,NaN,35.501183,-97.506973,"[{'label': 'display', 'lat': 35.50118300000000...",4025,73105,US,Oklahoma City,OK,United States,"[210 NE 31st St, Oklahoma City, OK 73105, Unit..."
3,513612d0e4b0bb44a517e4f7,Oklahoma Information Center MWC,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1605235001,False,NaN,NaN,35.434780,-97.400902,"[{'label': 'display', 'lat': 35.43477993651003...",10703,NaN,US,Oklahoma City,OK,United States,"[Oklahoma City, OK, United States]"
4,4da9d90e4df01c19b17bce51,Oklahoma Petroleum Information Center,[],v-1605235001,False,2020 Industrial Blvd,NaN,35.243014,-97.456105,"[{'label': 'display', 'lat': 35.243014, 'lng':...",25278,73069,US,Norman,OK,United States,"[2020 Industrial Blvd, Norman, OK 73069, Unite..."


In [33]:
## Clean the dataframe

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name','categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Downtown OKC Tourism Info Center,Miscellaneous Shop,"120 N Robinson, Ste 115",Park and Robinson,35.468561,-97.516351,"[{'label': 'display', 'lat': 35.468561, 'lng':...",481,73102,US,Oklahoma City,OK,United States,"[120 N Robinson, Ste 115 (Park and Robinson), ...",4c0692edb481d13a3c5819f0
1,Eden Bail Bond Information Center,None,805 Robert S Kerr Ave,NaN,35.470381,-97.526023,"[{'label': 'display', 'lat': 35.470381, 'lng':...",1322,73106,US,Oklahoma City,OK,United States,"[805 Robert S Kerr Ave, Oklahoma City, OK 7310...",569ca86b498e7bd114de7689
2,Oklahoma Tourism and Recreation Distribution ...,Tourist Information Center,210 NE 31st St,NaN,35.501183,-97.506973,"[{'label': 'display', 'lat': 35.50118300000000...",4025,73105,US,Oklahoma City,OK,United States,"[210 NE 31st St, Oklahoma City, OK 73105, Unit...",53c576a6498eccdf09559718
3,Oklahoma Information Center MWC,Government Building,NaN,NaN,35.434780,-97.400902,"[{'label': 'display', 'lat': 35.43477993651003...",10703,NaN,US,Oklahoma City,OK,United States,"[Oklahoma City, OK, United States]",513612d0e4b0bb44a517e4f7
4,Oklahoma Petroleum Information Center,None,2020 Industrial Blvd,NaN,35.243014,-97.456105,"[{'label': 'display', 'lat': 35.243014, 'lng':...",25278,73069,US,Norman,OK,United States,"[2020 Industrial Blvd, Norman, OK 73069, Unite...",4da9d90e4df01c19b17bce51
5,Oklahoma I 35 N Travel Information Center,Tourist Information Center,NaN,NaN,35.692531,-97.416360,"[{'label': 'display', 'lat': 35.69253067645059...",26756,73013,US,Edmond,OK,United States,"[Edmond, OK 73013, United States]",4e19f3cea8097d08b2458954
6,Norman Alcohol Information Center,Medical Center,NaN,NaN,35.215383,-97.443317,"[{'label': 'display', 'lat': 35.215383, 'lng':...",28531,73069,US,Norman,OK,United States,"[Norman, OK 73069, United States]",4d025e7cdc18b1f7bbddeb10
7,Comanche Nation Tourism Center,Tourist Information Center,NaN,NaN,34.602762,-98.376486,"[{'label': 'display', 'lat': 34.602762, 'lng':...",124165,NaN,US,Lawton,OK,United States,"[Lawton, OK, United States]",560ee9b7498e78507f2a487e


## Part 2: Visualize the dataframe

In [34]:
dataframe_filtered.name

0                     Downtown OKC Tourism Info Center
1                    Eden Bail Bond Information Center
2    Oklahoma Tourism  and Recreation Distribution ...
3                      Oklahoma Information Center MWC
4                Oklahoma Petroleum Information Center
5            Oklahoma I 35 N Travel Information Center
6                    Norman Alcohol Information Center
7                       Comanche Nation Tourism Center
Name: name, dtype: object

In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='73102',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Part 3: Results and Conclusion

As we can see, there is a sparse distribution of travel information centers in Oklahoma. There is a vast amount of opportunity to place new TICs in areas near or around popular destinations in the state. 